# SystemicTreatment to OMOP

In [ ]:
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta
import mysql.connector

In [ ]:
systemic_treatment_IDEA4RC = pd.read_csv("./IDEA4RC-data/systemicTreatmentIDEA4RC.csv")
systemic_treatment_IDEA4RC.head(5)

In [ ]:
# Conexión a la base de datos OMOP CDM
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

# Comando para hacer el mapeo (insercción de datos del IDEA4RC.csv a OMOP)
cur = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Systemic Treatment to Episode

Since it is not clear what to do with episde, we will try to create a new one, and if exists we will simply change the data

episode_type_concept_id??? I am using a value that is not correct.

In [1]:
df_tables=systemic_treatment_IDEA4RC
sql = """
INSERT INTO EPISODE (episode_id, person_id, episode_start_date, episode_end_date, episode_concept_id,episode_type_concept_id)
VALUES (%s, %s, %s, %s, %s,%s)
ON DUPLICATE KEY UPDATE
    episode_start_date = VALUES(episode_start_date),
    episode_end_date = VALUES(episode_end_date),
    episode_concept_id = VALUES(episode_concept_id),
    episode_type_concept_id=VALUES(episode_type_concept_id);
"""

queryPerson="""
    SELECT e.patient_id FROM cancer_episode e
    INNER JOIN episode_event v ON e.id=v.cancer_episode
    INNER JOIN systemic_treatment st ON st.episode_event=v.id
    WHERE st.id=%s
"""    


    
for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['id']))
    res=curIDEA.fetchone()
    person_id=res
    cur.execute(sql,(row['episode_event'], person_id, row['startdate'], row['enddate'],row['treatmentResponse'],row['treatmentResponse']))
    conn.commit()


SyntaxError: incomplete input (452453103.py, line 12)

### Systemic Treatment to Procedure Ocurrence

In [ ]:
df_tables=systemic_treatment_IDEA4RC
queryEVPO="""
    INSERT INTO omopcdm.episode_event (episode_id,event_id,episode_event_field_concept_id)
    VALUES (%s, %s, %s)
"""

sqlProcedure="""
    INSERT INTO omopcdm.procedure_ocurrence (procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_end_date,procedure_type_concept_id)
    VALUES (%s, %s, %s,%s,%s)
"""
query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM EpisodeEvent e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1

"""
columnsPO = [
    "Type of systemic treatment",
    "Intent",
    "Setting",
    "Chemotherapy info"
]

def toTableEpisodeEventProcedureOcurrence(idEpisode , idProcedureOcurrence):
    cur.execute(queryEVPO,(idEpisode,idProcedureOcurrence,1147082)) #1147082 or 1147810, not clear
    conn.commit()

for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Episode Event Reference']))
    res=curIDEA.fetchone()
    person_id=res
    procedure_date=row['Start date regimen changed']
    procedre_end_date=row['End date regimen changed']
    procedure_type_concept_id=0 #Still dont know what to do with this one
    for column in columnsPO:
        procedure_concept_id=row[column]
        newId = datetime.now().strftime('%Y%m%d%H%M%S') + str(uuid4())
        toTableEpisodeEventProcedureOcurrence(row['Episode Event reference'],newId)
        cur.execute(sqlProcedure,(newId, person_id, procedure_concept_id,procedure_date,procedure_end_date,procedure_type_concept_id))
        conn.commit()

### Systemic Treatment to Measurement

Measurement concept id???

meas_event_field_concept_id???

In [1]:
sqlMeasuement = """
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date, measurement_type_concept_id, value_as_number,measurement_event_id,meas_event_field_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s)
    """
measurement_type_concept_id=38000280
measurement_concept_id=0
for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Episode Event Reference']))
    res=curIDEA.fetchone()
    person_id=res
    date=row['Start date regimen changed']
    value_as_number=row['Number of cycles/ administrations']
    measurement_value=row['Number of cycles/ administrations']   #??? the same as value as number
    measurement_event_id=row['Episode Event reference']
    cur.execute(sqlMeasuement,(person_id, measurement_concept_id, date, measurement_type_concept_id,value_as_number,measurement_event_id,meas_event_field_concept_id))
    conn.commit()'Surgical specimen Mitotic count'

SyntaxError: invalid syntax (2571587072.py, line 16)